# Assignment 4
In this assignment you will be using the dataset released by The Department of Transportation. This dataset lists flights that occurred in 2015, along with other information such as delays, flight time etc.

In this assignment, you will be showing good practices to manipulate data using Python's most popular libraries to accomplish the following:

- cleaning data with pandas
- make specific changes with numpy
- handling date-related values with datetime

Note: please consider the flights departing from BOS, JFK, SFO and LAX.

Each question is equally weighted for the total grade.

In [6]:
import os
import pandas as pd
import pandas.api.types as ptypes
import numpy as np
import datetime as dt

# Retrieve data
flights_df_raw = pd.read_csv('assets/flights.csv', low_memory=False)
airports_df = pd.read_csv('assets/airports.csv')
airlines_df = pd.read_csv('assets/airlines.csv')

## Question 1: Data Preprocessing
For this question, perform the following:

- remove rows with missing values
- keep flights departing from airports (ORIGIN_AIRPORT) that we want to look at (BOS, JFK, SFO and LAX)
- filter out the flights that have more than 1 day delay (DEPARTURE_DELAY)
- convert FLIGHT_NUMBER column type to string
- SCHEDULED_DEPARTURE is coded as a float where the first two digits indicate the hour and the last two indicate the minutes. Convert this column to datetime format by combining existing columns DAY, MONTH, YEAR and SCHEDULED_DEPARTURE
- add IS_DELAYED column by considering any flight above 15 minutes delay (DEPARTURE_DELAY) are delayed, and any other flight is not delayed
- remove YEAR, MONTH, DAY columns

In [7]:
def data_preprocess(flights_df):
    # YOUR CODE HERE
    
    flight_drpval = flights_df_raw.dropna()
    
    #filering based on columns
    filtered_df = flight_drpval[flight_drpval['ORIGIN_AIRPORT'].isin(['BOS','JFK','SFO','LAX'])]
    
    ##filtering flights with more than a day delay
    notdelayed_df = filtered_df[filtered_df['DEPARTURE_DELAY'] <= 24*60]
    notdelayed_df['FLIGHT_NUMBER'] = notdelayed_df['FLIGHT_NUMBER'].astype(str)
    
   
    #converting to datetime format
    notdelayed_df['SCHEDULED_DEPARTURE'] = notdelayed_df['SCHEDULED_DEPARTURE'].astype(str).str.rjust(4,'0')
    notdelayed_df['SCHEDULED_DEPARTURE'] = pd.to_datetime(notdelayed_df['DAY'].astype(str) + '-' + 
                                                      notdelayed_df['MONTH'].astype(str)+'-'+ notdelayed_df['YEAR'].astype(str)+' ' + 
                                                     notdelayed_df['SCHEDULED_DEPARTURE'], format = '%d-%m-%Y %H%M')


    
    #add IS_DELAYED column
    notdelayed_df['IS_DELAYED'] = np.select([notdelayed_df['DEPARTURE_DELAY'] >= 15], [1],default=0)#np.where(notdelayed_df['DEPARTURE_DELAY']>=15,1,0)
    flights_df = notdelayed_df.drop(columns=['YEAR','MONTH','DAY'])

    
   
   # raise NotImplementedError()
    return flights_df

data_preprocess(flights_df_raw)

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,AIRLINE,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,IS_DELAYED
0,LAX,PBI,AA,2336,2015-01-01 00:10:00,-8.0,0
1,SFO,CLT,US,840,2015-01-01 00:20:00,-2.0,0
2,LAX,MIA,AA,258,2015-01-01 00:20:00,-5.0,0
4,SFO,MSP,DL,806,2015-01-01 00:25:00,-5.0,0
6,LAX,CLT,US,2013,2015-01-01 00:30:00,14.0,0
...,...,...,...,...,...,...,...
2966762,LAX,ORD,AA,219,2015-12-31 23:59:00,-1.0,0
2966766,LAX,BOS,B6,688,2015-12-31 23:59:00,-4.0,0
2966767,JFK,PSE,B6,745,2015-12-31 23:59:00,-4.0,0
2966768,JFK,SJU,B6,1503,2015-12-31 23:59:00,-9.0,0


In [8]:
flights_df = data_preprocess(flights_df_raw.copy())
assert len(flights_df) == 535744, "Q1: There should be 535744 observations in the flights dataframe"


## Question 2
Merge flights_df dataframe with airports_df dataframe and return the number of departing flights (*NUM_FLIGHTS*) per airport (*IATA_CODE*) across the year.

In [9]:
def flights_per_airport(flights_df_raw, airports_df):
    # YOUR CODE HERE
    
    new_df = flights_df
    new_df = pd.merge(flights_df, airports_df, how = 'outer', left_index = True, right_index = True)
    new_df

    df = new_df.groupby('ORIGIN_AIRPORT')['ORIGIN_AIRPORT'].count()
    df= df.to_frame()
    df = df.rename(columns={"ORIGIN_AIRPORT":"NUM_FLIGHTS"})
    df.reset_index()
    
    print(df.shape)
    print(df.columns[0])
    print(df.loc["BOS","NUM_FLIGHTS"])
    df
    
   # raise NotImplementedError()
    return df
flights_per_airport(flights_df_raw.copy(), airports_df.copy()).shape

(4, 1)
NUM_FLIGHTS
105276


(4, 1)

In [7]:
num_flights_df=flights_per_airport(flights_df_raw.copy(), airports_df.copy())

assert num_flights_df.shape==(4,1), "Shape of DataFrame should be (4,1)"
assert num_flights_df.columns[0]=='NUM_FLIGHTS', "DataFrame should have a column which is called NUM_FLIGHTS"
assert num_flights_df.loc["BOS", "NUM_FLIGHTS"] == 105276, "The NUM_FLIGHTS for BOS is wrong"


(4, 1)
NUM_FLIGHTS
105276


## Question 3
For this question, find the top three airline names which have high number of flights and the least percentage of delay compared to other airlines. The result should be a dataframe which has three columns *AIRLINE_NAME*, *NUM_FLIGHTS* and *PERC_DELAY*.

Hint:
- percentage of delay for each airline is obtained using groupby and apply methods
- merge flights_df with airlines_df to get the names of top three airlines

In [10]:
def top_three_airlines(flights_df_raw, airlines_df):
    # YOUR CODE HERE
    
    df = (
    flights_df
 
   .groupby(['AIRLINE'])
   .apply(lambda x: pd.Series({
      'NUM_FLIGHTS': len(x['FLIGHT_NUMBER']),
      'PERC_DELAY': round(np.average(x['IS_DELAYED']), 4)
   }))
   .reset_index()
  
   .assign(rank_size=lambda x: x['NUM_FLIGHTS'].rank(method='first', ascending=False))
   [lambda x: x['rank_size'] <= 3]
   .drop('rank_size', axis=1)
  
   .merge(airlines_df, left_on='AIRLINE', right_on='IATA_CODE')
   .drop(['AIRLINE_x','IATA_CODE'], axis = 1)    
   .rename(columns = {"AIRLINE_y":"AIRLINE_NAME"})   
   .sort_values('NUM_FLIGHTS',ascending = False) 
   .reset_index(drop=True)     
)
    
   # raise NotImplementedError()
    return df
top_three_airlines(flights_df_raw.copy(), airlines_df.copy())

,NUM_FLIGHTS,PERC_DELAY,AIRLINE_NAME
0,86562.0,0.2327,United Air Lines Inc.
1,85920.0,0.2080,JetBlue Airways
2,77024.0,0.1535,American Airlines Inc.


In [11]:
top_three_airlines_df = top_three_airlines(flights_df_raw.copy(), airlines_df.copy())

assert sorted(list(top_three_airlines_df.columns)) == sorted(['NUM_FLIGHTS', 'PERC_DELAY', 'AIRLINE_NAME']), "Dataframe doesn't have required columns"
assert top_three_airlines_df.loc[0, 'AIRLINE_NAME'] == 'United Air Lines Inc.', "Top airline name doesn't match"


## Question 4
For this question, obtain the monthly percentage of delays for each *ORIGIN_AIRPORT*.

Example Result:

         MONTH     BOS     JFK     LAX     SFO
    0   January  0.1902  0.2257  0.1738  0.xxxx
    1  February  0.3248  0.xxxx  0.xxxx  0.xxxx
    2     March  0.1984  0.xxxx  0.xxxx  0.xxxx
    3     April  0.xxxx  0.xxxx  0.xxxx  0.xxxx

In [17]:
def monthly_airport_delays(flights_df_raw):
    # YOUR CODE HERE
    
    df = (
    flights_df
    
    .assign(
        MONTH=lambda x: x['SCHEDULED_DEPARTURE'].dt.strftime('%B'),
        MONTH_NUM=lambda x: x['SCHEDULED_DEPARTURE'].dt.strftime('%m').astype(int)
    )
    
    .groupby(['ORIGIN_AIRPORT', 'MONTH', 'MONTH_NUM'])
    .apply(lambda x: pd.Series({
        'PERC_DELAY': round(np.average(x['IS_DELAYED']), 4)
    }))
    .reset_index()
    
    .pivot_table(
        columns='ORIGIN_AIRPORT',
        values='PERC_DELAY',
        index=['MONTH', 'MONTH_NUM'],
        aggfunc=np.sum
    )
    .rename_axis(None, axis=1)
    .reset_index()
    
    .sort_values(['MONTH_NUM'])
    .drop(['MONTH_NUM'], axis=1)
    .reset_index(drop=True)
)
    
   # raise NotImplementedError()
    return df
monthly_airport_delays(flights_df_raw.copy())

,MONTH,BOS,JFK,LAX,SFO
0,January,0.1902,0.2257,0.1738,0.2001
1,February,0.3248,0.3174,0.1978,0.2222
2,March,0.1984,0.2736,0.2246,0.1770
3,April,0.1553,0.2020,0.1855,0.1756
4,May,0.1552,0.1552,0.1990,0.2297
5,June,0.2032,0.1938,0.2474,0.2374
6,July,0.1968,0.2019,0.2772,0.2430
7,August,0.1988,0.1985,0.2289,0.2118
8,September,0.1274,0.1506,0.1486,0.1399
9,November,0.1177,0.1383,0.1741,0.1565


In [16]:
monthly_airport_delays_df = monthly_airport_delays(flights_df_raw.copy())
